# Lab 5: Single Agent with MCP Server Integration

Build an AI agent using **Microsoft Agent Framework** that connects to your **MCP servers** from Lab 4!

## Learning Objectives

By the end of this lab, you will:
1. Connect an Azure AI Agent to a **local MCP server**
2. Connect an Azure AI Agent to a **remote (deployed) MCP server**
3. Understand the difference between **HostedMCPTool** vs **MCPStreamableHTTPTool**
4. Implement **user approval workflows** for secure tool execution
5. Build a complete **Gradio chat interface** with MCP-powered tools

## Prerequisites

| Requirement | Setup |
|------------|-------|
| Python environment | Run `make start_env` from project root |
| Azure CLI logged in | Run `az login` in terminal |
| Azure AI Foundry Project | [Create at ai.azure.com](https://ai.azure.com) |
| Model deployment | Deploy GPT-4o in your AI Foundry project |
| **Lab 4 completed** | MCP server deployed to Azure |

## Environment Variables

Create a `.env` file in the project root:

```bash
# Required
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o

# From Lab 4 - MCP Server
MCP_SERVER_URL=https://func-api-xxxx.azurewebsites.net/runtime/webhooks/mcp
MCP_SERVER_KEY=your-mcp-extension-key

# Local MCP (optional - for local development)
MCP_LOCAL_URL=http://localhost:7071/runtime/webhooks/mcp
```

## Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                      Microsoft Agent Framework                               │
│                                                                             │
│  ┌─────────────────────────────────────────────────────────────────────┐    │
│  │                    AzureAIAgentsProvider                            │    │
│  │                                                                     │    │
│  │   ┌─────────┐         ┌──────────────────────────────────┐         │    │
│  │   │  Agent  │────────▶│     MCP Tool Integration         │         │    │
│  │   │  (LLM)  │◀────────│                                  │         │    │
│  │   └────┬────┘         │  ┌────────────────────────────┐  │         │    │
│  │        │              │  │ MCPStreamableHTTPTool      │  │         │    │
│  │        │              │  │ (Run-level, manual connect)│  │         │    │
│  │        │              │  └────────────────────────────┘  │         │    │
│  │        │              │  ┌────────────────────────────┐  │         │    │
│  │        │              │  │ HostedMCPTool              │  │         │    │
│  │        │              │  │ (Agent-level, auto-managed)│  │         │    │
│  │        │              │  └────────────────────────────┘  │         │    │
│  │        │              └──────────────────────────────────┘         │    │
│  └────────┼───────────────────────────────────────────────────────────┘    │
└───────────┼─────────────────────────────────────────────────────────────────┘
            │
            │ SSE + JSON-RPC 2.0 (MCP Protocol)
            │
   ┌────────┴────────┐                          ┌────────────────────┐
   │  Local Server   │                          │  Remote Server     │
   │  (func start)   │                          │  (Azure Functions) │
   │  :7071          │                          │  *.azurewebsites.net│
   └─────────────────┘                          └────────────────────┘
         │                                              │
         ▼                                              ▼
   ┌─────────────────────────────────────────────────────────────────┐
   │                     MCP Tools (from Lab 4)                      │
   │  • hello_mcp      - Health check                                │
   │  • analyze_data   - Statistical analysis                        │
   │  • save_snippet   - Save code snippets                          │
   │  • get_snippet    - Retrieve snippets                           │
   │  • list_snippets  - List all snippets                           │
   └─────────────────────────────────────────────────────────────────┘
```

**Key Difference from Lab 3:**
- Lab 3: Tools defined as **Python functions** (local, Logic App HTTP, Azure Function HTTP)
- Lab 5: Tools discovered from **MCP servers** (standardized protocol, auto-discovery)

## MCP Tool Integration Patterns

| Pattern | Class | Connection | Best For |
|---------|-------|------------|----------|
| **Agent-level** | `HostedMCPTool` | Auto-managed by agent | Simple use, less control |
| **Run-level** | `MCPStreamableHTTPTool` | Manual context manager | More control, dynamic tools |

### Agent-Level (HostedMCPTool)
```python
agent = await provider.create_agent(
    tools=HostedMCPTool(name="my-mcp", url="https://...")
)
async with agent:
    result = await agent.run(query)
```

### Run-Level (MCPStreamableHTTPTool)
```python
async with MCPStreamableHTTPTool(name="my-mcp", url="https://...") as mcp:
    result = await agent.run(query, tools=mcp)
```

---

## Step 1: Setup and Environment

In [ ]:
import os
import sys
from pathlib import Path

from dotenv import load_dotenv

# Ensure Azure CLI is in PATH (Homebrew installs to /opt/homebrew/bin on Apple Silicon)
homebrew_paths = ["/opt/homebrew/bin", "/usr/local/bin"]
current_path = os.environ.get("PATH", "")
for p in homebrew_paths:
    if p not in current_path:
        os.environ["PATH"] = p + ":" + current_path
        current_path = os.environ["PATH"]

# Verify az is accessible
import shutil
az_path = shutil.which("az")
if az_path:
    print(f"✅ Azure CLI found: {az_path}")
else:
    print("⚠️  Azure CLI not found. Run: brew install azure-cli && az login")

# Load environment variables from project root
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from {env_path.resolve()}")
else:
    load_dotenv()  # Try current directory
    print("⚠️  No .env found in parent directory, tried current directory")

# Verify required env vars
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing = [v for v in required_vars if not os.getenv(v)]
if missing:
    print(f"❌ Missing environment variables: {missing}")
    print("   Please create a .env file in the project root with:")
    print("   AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com")
    print("   AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o")
else:
    print("✅ Environment configured")
    print(f"   Project: {os.getenv('AZURE_AI_PROJECT_ENDPOINT')[:50]}...")
    print(f"   Model: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")

# Check MCP server configuration
print("\n🔧 MCP Server Configuration:")
mcp_remote_url = os.getenv("MCP_SERVER_URL")
mcp_remote_key = os.getenv("MCP_SERVER_KEY")
mcp_local_url = os.getenv("MCP_LOCAL_URL", "http://localhost:7071/runtime/webhooks/mcp")

if mcp_remote_url:
    print(f"   ✅ Remote MCP: {mcp_remote_url[:50]}...")
else:
    print("   ⚠️  MCP_SERVER_URL not set (remote MCP won't work)")
    
print(f"   📍 Local MCP: {mcp_local_url}")

## Step 2: Install Required Packages

The `agent-framework` package provides the MCP integration classes.

In [ ]:
# Install agent-framework if not already installed
import subprocess

try:
    from agent_framework import MCPStreamableHTTPTool, HostedMCPTool
    print("✅ agent_framework already installed")
except ImportError:
    print("📦 Installing agent-framework...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "agent-framework", "-q"])
    from agent_framework import MCPStreamableHTTPTool, HostedMCPTool
    print("✅ agent_framework installed successfully")

# Verify imports
from agent_framework import AgentProtocol, AgentResponse, AgentThread, HostedMCPTool, MCPStreamableHTTPTool
from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import DefaultAzureCredential, AzureCliCredential

print("\n✅ All imports successful:")
print("   • AgentProtocol, AgentResponse, AgentThread")
print("   • HostedMCPTool, MCPStreamableHTTPTool")
print("   • AzureAIAgentsProvider")
print("   • DefaultAzureCredential, AzureCliCredential")

---

## Step 3: Connect to Local MCP Server

First, let's connect to the **local MCP server** you created in Lab 4.

### ⚠️ Before running this cell:
1. Open a terminal in the `src/tool_registry/mcps/src` directory
2. Start Azurite: `azurite --silent &`
3. Start the MCP server: `func start`
4. Verify it's running at http://localhost:7071

In [ ]:
import asyncio
from typing import Any

# Use MCPStreamableHTTPTool for run-level tool integration
# This gives us more control over the connection lifecycle

async def test_local_mcp_agent():
    """Test agent with local MCP server (tools defined at run level)."""
    
    print("🔌 Connecting to local MCP server...")
    print(f"   URL: {mcp_local_url}")
    print()
    
    async with (
        DefaultAzureCredential() as credential,
        # MCPStreamableHTTPTool connects to the MCP server
        # It auto-discovers available tools via the MCP protocol
        MCPStreamableHTTPTool(
            name="lab4-local-mcp",
            url=mcp_local_url,
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent WITHOUT tools (we'll pass them at run time)
        agent = await provider.create_agent(
            name="MCPAssistant",
            instructions="""You are a helpful assistant with access to MCP tools.
            
Available MCP Tools (from Lab 4):
- hello_mcp: Health check to verify MCP connectivity
- analyze_data: Analyze numbers and return statistics (count, sum, mean, min, max)
- save_snippet: Save a code snippet with a name
- get_snippet: Retrieve a saved snippet by name
- list_snippets: List all saved snippet names

When asked to analyze data, use the analyze_data tool with JSON array format.
When asked to save/get code, use the snippet tools.""",
        )
        
        print(f"✅ Agent created: {agent.name}")
        print(f"   MCP Server: lab4-local-mcp")
        print()
        
        # Test queries
        test_queries = [
            "Say hello using the MCP server",
            "Analyze these numbers: [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]",
            "Save this Python code as 'hello_world': print('Hello, Northwestern!')",
            "List all saved snippets",
        ]
        
        for query in test_queries:
            print("=" * 70)
            print(f"📤 User: {query}")
            print("-" * 70)
            # Pass MCP tools at run time
            result = await agent.run(query, tools=mcp_server)
            print(f"🤖 Assistant: {result}")
            print()

# Run the test
await test_local_mcp_agent()

---

## Step 4: Connect to Remote MCP Server (Azure)

Now let's connect to the **deployed MCP server** in Azure that you created in Lab 4.

### Remote Server Authentication

Remote Azure Functions MCP servers require the `x-functions-key` header:

```python
MCPStreamableHTTPTool(
    url="https://func-xxx.azurewebsites.net/runtime/webhooks/mcp",
    headers={"x-functions-key": "your-mcp-extension-key"}
)
```

In [ ]:
async def test_remote_mcp_agent():
    """Test agent with remote (Azure) MCP server."""
    
    if not mcp_remote_url or not mcp_remote_key:
        print("❌ Remote MCP not configured!")
        print("   Set MCP_SERVER_URL and MCP_SERVER_KEY in your .env file")
        print("   Get these values from Lab 4 deployment.")
        return
    
    print("🔌 Connecting to remote MCP server (Azure)...")
    print(f"   URL: {mcp_remote_url[:60]}...")
    print()
    
    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-remote-mcp",
            url=mcp_remote_url,
            headers={"x-functions-key": mcp_remote_key},  # Auth header for Azure Functions
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="RemoteMCPAssistant",
            instructions="""You are a helpful assistant with access to remote MCP tools deployed on Azure.
            
Available MCP Tools:
- hello_mcp: Health check
- analyze_data: Statistical analysis
- save_snippet/get_snippet/list_snippets: Code snippet storage

The tools are running on Azure Functions (serverless compute).""",
        )
        
        print(f"✅ Agent created: {agent.name}")
        print(f"   MCP Server: lab4-remote-mcp (Azure)")
        print()
        
        # Test with complex query
        query = """Please do the following:
        1. Say hello to verify MCP connectivity
        2. Analyze these exam scores: [85, 92, 78, 95, 88, 76, 91, 83, 89, 94]
        3. Save a summary as 'exam_summary' snippet
        """
        
        print("=" * 70)
        print(f"📤 User: {query}")
        print("=" * 70)
        result = await agent.run(query, tools=mcp_server)
        print(f"\n🤖 Assistant: {result}")

# Run the test
await test_remote_mcp_agent()

---

## Step 5: Agent-Level MCP Integration (HostedMCPTool)

An alternative pattern is **agent-level** tool integration using `HostedMCPTool`.

### Key Differences

| Aspect | MCPStreamableHTTPTool | HostedMCPTool |
|--------|----------------------|---------------|
| Connection | Manual (`async with`) | Auto-managed by agent |
| When to use | Run-level tools | Agent-level tools |
| Control | More explicit | Less boilerplate |
| Tool discovery | At connection time | At agent creation |

With `HostedMCPTool`, you define tools when creating the agent, and the agent manages the MCP connection:

In [ ]:
async def test_hosted_mcp_tool():
    """Test agent with HostedMCPTool (agent-level integration)."""
    
    print("🔌 Using HostedMCPTool (agent-level integration)...")
    print()
    
    async with (
        DefaultAzureCredential() as credential,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent WITH tools defined at creation time
        agent = await provider.create_agent(
            name="HostedMCPAssistant",
            instructions="""You are a helpful assistant with MCP tools.
Use hello_mcp for health checks, analyze_data for statistics.""",
            tools=HostedMCPTool(
                name="lab4-mcp",
                url=mcp_local_url,  # Using local for demo; use mcp_remote_url + headers for Azure
            ),
        )
        
        print(f"✅ Agent created with HostedMCPTool")
        
        # Use agent as context manager to connect MCP tools
        async with agent:
            query = "Say hello and then analyze: [1, 2, 3, 4, 5]"
            print(f"\n📤 User: {query}")
            print("-" * 50)
            result = await agent.run(query)
            print(f"🤖 Assistant: {result}")

# Run the test
await test_hosted_mcp_tool()

---

## Step 6: User Approval Workflow (Security)

For **sensitive operations**, you can implement a **user approval workflow**.

This is especially important when:
- Tools can modify data (save_snippet, delete operations)
- Tools access external systems
- You want human-in-the-loop verification

### How it Works

```
User Query ──▶ Agent ──▶ Tool Call Request
                              │
                              ▼
                    User Approval Prompt
                    "Approve save_snippet?"
                              │
                    ┌────────┴────────┐
                    ▼                 ▼
                 Approve          Reject
                    │                 │
                    ▼                 ▼
               Execute Tool     Skip Tool
                    │                 │
                    └────────┬────────┘
                             ▼
                      Final Response
```

In [ ]:
from agent_framework import ChatMessage

async def handle_approvals_with_thread(
    query: str,
    agent: AgentProtocol,
    thread: AgentThread,
    auto_approve: bool = False,
) -> AgentResponse:
    """
    Process agent response with user approval for tool calls.
    
    Args:
        query: The user's question
        agent: The AI agent
        thread: Conversation thread for context
        auto_approve: If True, approve all tool calls automatically
    
    Returns:
        Final agent response after all tool calls processed
    """
    result = await agent.run(query, thread=thread, store=True)
    
    # Check if agent requested tool calls that need approval
    while len(result.user_input_requests) > 0:
        new_input: list[Any] = []
        
        for user_input_needed in result.user_input_requests:
            # Display tool call details
            func_name = user_input_needed.function_call.name
            func_args = user_input_needed.function_call.arguments
            
            print(f"\n🔧 Tool Call Request:")
            print(f"   Function: {func_name}")
            print(f"   Arguments: {func_args}")
            
            if auto_approve:
                approved = True
                print("   ✅ Auto-approved")
            else:
                # In notebook, we'll use auto_approve
                # In production, you'd use: input("Approve? (y/n): ").lower() == 'y'
                user_input = input("   Approve this tool call? (y/n): ").strip().lower()
                approved = user_input == 'y'
                print(f"   {'✅ Approved' if approved else '❌ Rejected'}")
            
            # Create approval/rejection response
            new_input.append(
                ChatMessage(
                    role="user",
                    contents=[user_input_needed.create_response(approved)],
                )
            )
        
        # Continue agent execution with approval responses
        result = await agent.run(new_input, thread=thread, store=True)
    
    return result


async def test_approval_workflow():
    """Test agent with user approval workflow."""
    
    print("🔐 Testing User Approval Workflow")
    print("=" * 50)
    
    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-mcp",
            url=mcp_local_url,
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="SecureMCPAssistant",
            instructions="""You are a helpful assistant with MCP tools.
Use the tools when needed to help the user.""",
            tools=mcp_server,
        )
        
        thread = agent.get_new_thread()
        
        # Query that triggers multiple tool calls
        query = "Save a snippet called 'greeting' with content 'Hello from Lab 5!'"
        
        print(f"\n📤 User: {query}")
        print("-" * 50)
        
        # Use auto_approve=True for notebook demo
        # Set to False for interactive approval
        result = await handle_approvals_with_thread(query, agent, thread, auto_approve=True)
        
        print(f"\n🤖 Assistant: {result}")

# Run the test
await test_approval_workflow()

---

## Step 7: Conversation with Thread History

Maintain conversation context across multiple queries using `AgentThread`.

This allows the agent to:
- Remember previous questions
- Reference earlier tool results
- Build on prior context

In [ ]:
async def test_conversation_with_history():
    """Test multi-turn conversation with thread history."""
    
    print("💬 Multi-Turn Conversation with MCP Tools")
    print("=" * 60)
    
    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-mcp",
            url=mcp_local_url,
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="ConversationAgent",
            instructions="""You are a helpful data analyst assistant.
You can analyze data and save results as snippets.
Remember the conversation context.""",
            tools=mcp_server,
        )
        
        # Create a thread to maintain conversation history
        thread = agent.get_new_thread()
        print(f"📝 New conversation thread created\n")
        
        # Multi-turn conversation
        conversation = [
            "Analyze these test scores: [85, 92, 78, 95, 88]",
            "What was the mean score you just calculated?",
            "Save those results as a snippet called 'test_scores_analysis'",
            "What snippets do we have saved now?",
        ]
        
        for i, query in enumerate(conversation, 1):
            print(f"\n{'─' * 60}")
            print(f"📤 Turn {i}: {query}")
            print(f"{'─' * 60}")
            
            # Pass the same thread to maintain context
            result = await agent.run(query, thread=thread, tools=mcp_server, store=True)
            print(f"🤖 {result}")

# Run the test
await test_conversation_with_history()

---

## Step 8: Gradio Chat Interface 🎨

Build a complete chat interface that connects to your MCP server!

**Features:**
- Toggle between local and remote MCP servers
- Conversation history maintained
- Example queries for easy testing

In [ ]:
import gradio as gr
import nest_asyncio
from agent_framework import MCPStreamableHTTPTool
from agent_framework.azure import AzureAIAgentsProvider
from azure.identity.aio import DefaultAzureCredential

# Enable nested event loops for Jupyter
nest_asyncio.apply()

# Session state for agent
class MCPAgentSession:
    def __init__(self):
        self.agent = None
        self.thread = None
        self.provider = None
        self.credential = None
        self.mcp_server = None
        self.initialized = False
        self.use_remote = False
    
    async def initialize(self, use_remote: bool = False):
        """Initialize agent with local or remote MCP server."""
        # Cleanup previous session if switching servers
        if self.initialized and self.use_remote != use_remote:
            await self.cleanup()
        
        if not self.initialized:
            self.use_remote = use_remote
            self.credential = DefaultAzureCredential()
            self.provider = AzureAIAgentsProvider(credential=self.credential)
            await self.provider.__aenter__()
            
            # Configure MCP server connection
            if use_remote and mcp_remote_url and mcp_remote_key:
                mcp_url = mcp_remote_url
                mcp_headers = {"x-functions-key": mcp_remote_key}
                server_type = "Azure (remote)"
            else:
                mcp_url = mcp_local_url
                mcp_headers = {}
                server_type = "Local"
            
            self.mcp_server = MCPStreamableHTTPTool(
                name="lab4-mcp",
                url=mcp_url,
                headers=mcp_headers if mcp_headers else None,
            )
            await self.mcp_server.__aenter__()
            
            self.agent = await self.provider.create_agent(
                name="MCPChatAssistant",
                instructions=f"""You are a helpful assistant with MCP tools ({server_type} server).

Available MCP Tools:
📡 hello_mcp - Health check to verify MCP connectivity
📊 analyze_data - Analyze numbers (pass as JSON array like [1,2,3])
💾 save_snippet - Save code with a name
📖 get_snippet - Retrieve saved code
📋 list_snippets - List all saved snippets

Be helpful and use tools when appropriate.""",
                tools=self.mcp_server,
            )
            self.thread = self.agent.get_new_thread()
            self.initialized = True
            print(f"✅ Agent initialized with {server_type} MCP server")
    
    async def chat(self, message: str, use_remote: bool = False) -> str:
        """Process chat message with MCP-enabled agent."""
        try:
            await self.initialize(use_remote)
            response = await self.agent.run(
                message,
                thread=self.thread,
                tools=self.mcp_server,
                store=True,
            )
            return str(response)
        except Exception as e:
            import traceback
            traceback.print_exc()
            return f"❌ Error: {str(e)}"
    
    async def cleanup(self):
        """Cleanup resources."""
        if self.mcp_server:
            await self.mcp_server.__aexit__(None, None, None)
        if self.provider:
            await self.provider.__aexit__(None, None, None)
        if self.credential:
            await self.credential.__aexit__(None, None, None)
        self.initialized = False
    
    def reset(self):
        """Reset conversation thread."""
        if self.agent:
            self.thread = self.agent.get_new_thread()

# Create session
session = MCPAgentSession()

# Gradio response function
async def respond(message: str, history: list, use_remote: bool):
    """Handle chat messages."""
    if not message.strip():
        return history, ""
    
    bot_response = await session.chat(message, use_remote)
    
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": bot_response})
    return history, ""

def clear_chat():
    """Clear chat and reset thread."""
    session.reset()
    return []

# Build Gradio interface
with gr.Blocks(title="MCP Agent Chat - Lab 5") as demo:
    gr.Markdown("""
    # 🔌 MCP Agent Chat (Lab 5)
    
    Chat with an AI agent connected to your **MCP server from Lab 4**!
    
    **Available Tools:**
    - 📡 `hello_mcp` — Health check
    - 📊 `analyze_data` — Statistical analysis (use JSON arrays: [1,2,3])
    - 💾 `save_snippet` / `get_snippet` / `list_snippets` — Code storage
    """)
    
    with gr.Row():
        use_remote = gr.Checkbox(
            label="Use Remote (Azure) MCP Server",
            value=False,
            info="Toggle to switch between local and remote MCP"
        )
    
    chatbot = gr.Chatbot(
        height=450,
        placeholder="Ask me anything! I have MCP tools for data analysis and snippet storage.",
    )
    
    msg = gr.Textbox(
        placeholder="Type your message... (e.g., 'Analyze [85, 92, 78, 95, 88]')",
        label="Your Message",
        lines=2,
    )
    
    with gr.Row():
        submit = gr.Button("Send 📤", variant="primary")
        clear = gr.Button("Clear 🗑️")
    
    gr.Examples(
        examples=[
            "Say hello to verify MCP connectivity",
            "Analyze these test scores: [85, 92, 78, 95, 88, 76, 91, 83]",
            "Save this Python code as 'fibonacci': def fib(n): return n if n < 2 else fib(n-1) + fib(n-2)",
            "List all saved snippets",
            "Get the 'fibonacci' snippet",
        ],
        inputs=msg,
    )
    
    # Event handlers
    msg.submit(respond, [msg, chatbot, use_remote], [chatbot, msg])
    submit.click(respond, [msg, chatbot, use_remote], [chatbot, msg])
    clear.click(clear_chat, outputs=[chatbot])

# Launch
print("🚀 Launching MCP Agent Chat...")
print(f"   Local MCP: {mcp_local_url}")
if mcp_remote_url:
    print(f"   Remote MCP: {mcp_remote_url[:50]}...")
demo.launch(share=False, height=700)

---

## Step 9: Combining MCP with Local Tools

You can combine **MCP tools** with **local Python functions** for a hybrid approach.

This is useful when:
- Some tools need to be MCP (shared across systems)
- Some tools are specific to this agent (local functions)

In [ ]:
from typing import Annotated
from pydantic import Field
from datetime import datetime

# Local function tool (not MCP)
def get_current_time() -> str:
    """Get the current date and time."""
    now = datetime.now()
    return f"📅 Current time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

def calculate_grade(
    score: Annotated[float, Field(description="The score achieved")],
    max_score: Annotated[float, Field(description="Maximum possible score")],
) -> str:
    """Calculate letter grade from score."""
    percentage = (score / max_score) * 100
    
    if percentage >= 93:
        letter = "A"
    elif percentage >= 90:
        letter = "A-"
    elif percentage >= 87:
        letter = "B+"
    elif percentage >= 83:
        letter = "B"
    elif percentage >= 80:
        letter = "B-"
    elif percentage >= 77:
        letter = "C+"
    elif percentage >= 73:
        letter = "C"
    elif percentage >= 70:
        letter = "C-"
    elif percentage >= 60:
        letter = "D"
    else:
        letter = "F"
    
    return f"📝 Score: {score}/{max_score} = {percentage:.1f}% → Grade: {letter}"


async def test_hybrid_tools():
    """Test agent with both MCP tools and local functions."""
    
    print("🔀 Hybrid Agent: MCP Tools + Local Functions")
    print("=" * 60)
    
    async with (
        DefaultAzureCredential() as credential,
        MCPStreamableHTTPTool(
            name="lab4-mcp",
            url=mcp_local_url,
        ) as mcp_server,
        AzureAIAgentsProvider(credential=credential) as provider,
    ):
        # Create agent with both local functions and MCP tools
        agent = await provider.create_agent(
            name="HybridAssistant",
            instructions="""You are a helpful academic assistant with both local and MCP tools.

Local Tools:
- get_current_time: Get current date/time
- calculate_grade: Convert score to letter grade

MCP Tools (from Lab 4):
- analyze_data: Statistical analysis of numbers
- save_snippet/get_snippet/list_snippets: Code storage

Use the appropriate tool for each task.""",
            tools=[
                get_current_time,    # Local function
                calculate_grade,     # Local function
                mcp_server,          # MCP tools (multiple tools from server)
            ],
        )
        
        print(f"✅ Hybrid agent created with local + MCP tools\n")
        
        # Test query using multiple tool types
        query = """Please help me with the following:
        1. What time is it now?
        2. I got 87 out of 100 on my exam - what's my letter grade?
        3. Analyze these quiz scores: [85, 92, 78, 95, 88]
        """
        
        print(f"📤 User: {query}")
        print("=" * 60)
        result = await agent.run(query)
        print(f"\n🤖 Assistant: {result}")

# Run the test
await test_hybrid_tools()

---

## Summary

In this lab, you learned how to integrate **MCP servers** with **Azure AI Agents**!

### Key Concepts

| Concept | Description |
|---------|-------------|
| **MCPStreamableHTTPTool** | Run-level MCP connection with explicit lifecycle |
| **HostedMCPTool** | Agent-level MCP with auto-managed connection |
| **User Approval** | Security workflow for sensitive tool calls |
| **Thread History** | Maintain conversation context across turns |
| **Hybrid Tools** | Combine MCP + local functions in one agent |

### MCP Integration Patterns

```python
# Pattern 1: Run-level tools (more control)
async with MCPStreamableHTTPTool(url="...") as mcp:
    result = await agent.run(query, tools=mcp)

# Pattern 2: Agent-level tools (simpler)
agent = await provider.create_agent(tools=HostedMCPTool(url="..."))
async with agent:
    result = await agent.run(query)

# Pattern 3: Hybrid (MCP + local)
agent = await provider.create_agent(
    tools=[local_func, mcp_server]
)
```

### Connection Settings

| Server Type | URL | Authentication |
|-------------|-----|----------------|
| Local | `http://localhost:7071/runtime/webhooks/mcp` | None |
| Azure | `https://func-xxx.azurewebsites.net/runtime/webhooks/mcp` | `x-functions-key` header |

### What's Next?

- **Lab 6**: Multi-agent systems with specialized agents
- **Production**: Deploy your MCP-enabled agent as an API
- **Extend**: Add more MCP tools to your Lab 4 server

---

**🎉 Congratulations!** You've built an AI agent that can use MCP tools from both local and cloud servers!